# Sentiment Analysis

## Imports & Configs

In [1]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import Dataset , DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import nltk
nltk.download('punkt')

# setup device agnostic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


device(type='cpu')

## Load Dataset (IMDB from huggingface)

In [2]:
from datasets import load_dataset
dataset = load_dataset("imdb")

train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts  = dataset['test']['text']
test_labels  = dataset['test']['label']





README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
print(f'Train size: {len(train_texts)}')
print(f'Test size: {len(test_texts)}')


Train size: 25000
Test size: 25000


## Preprocessing

In [ ]:
def clean_text(string):
  string = string.lower()
  string = re.sub(r"https\S+","",string)
  string = re.sub(r"@\w+", "", string)
  string = re.sub(r"[^a-z0-9\s']"," ",string)
  string = re.sub(r"\s+"," ",string).strip()

  return string

